In [ ]:
from pathlib import Path
from mpunet.hyperparameters import YAMLHParams
from mpunet.bin.train import get_logger
import numpy as np
import os

In [ ]:
os.chdir('/Users/px/GoogleDrive/MultiPlanarUNet')

project_dir = './my_project'
logger = get_logger(project_dir, True)
logger("Fitting model in path:\n%s" % project_dir)

hparams = YAMLHParams(project_dir + "/train_hparams.yaml", logger=logger)

for key, value in hparams.items():
    print(f'{key}: {value}')

In [ ]:
from mpunet.preprocessing import get_preprocessing_func, data_preparation_funcs
func = get_preprocessing_func(hparams["build"].get("model_class_name"))

# hparams['fit']['max_loaded'] = args.max_loaded_images
# hparams['fit']['num_access'] = args.num_access
# train, val, hparams = func(hparams=hparams,
#                            logger=logger,
#                            just_one=False,
#                            no_val=False,
#                            continue_training=False,
#                            base_path=project_dir)

In [ ]:
from mpunet.preprocessing import data_preparation_funcs as dpf
func = dpf.prepare_for_multi_view_unet

In [ ]:
from mpunet.image.image_pair_loader import ImagePairLoader
train_data = ImagePairLoader(logger=logger, **hparams["train_data"])
val_data = ImagePairLoader(logger=logger, **hparams["val_data"])


In [ ]:
train_data._id_to_image

In [ ]:
data_dir = Path("./").absolute()
images_path = data_dir / 'images'
os.path.abspath("./")

In [ ]:
images = train_data.images

In [ ]:
images[1]

##%
for i in train_data:
    print(i)

In [ ]:
train_data

In [ ]:
train_data.get_id_to_images_dict()

In [ ]:
{image.identifier: image for image in train_data.images}

In [ ]:
from mpunet.image.queue.utils import get_data_queues
max_loaded = hparams['fit'].get('max_loaded')
train_queue, val_queue = get_data_queues(
    train_dataset=train_data,
    val_dataset=val_data,
    train_queue_type='limitation' if max_loaded else "eager",
    val_queue_type='eager',
    max_loaded=max_loaded,
    num_access_before_reload=hparams['fit'].get('num_access'),
    logger=logger
)

In [ ]:
[i for i in train_queue]

In [ ]:
'''
ok, now need to run load_or_create_views,
but load np file first to see what happens
'''

In [ ]:
view_path = '/Users/px/GoogleDrive/' \
            'MultiPlanarUNet/my_project/views.npz'

In [ ]:
views = np.load(view_path)

In [ ]:
views_data = views['arr_0']

In [ ]:
hparams["fit"]["views"]

In [ ]:
hparams["fit"]["views"] = views_data

In [ ]:
from mpunet.preprocessing.data_preparation_funcs import get_sequencers
import mpunet.preprocessing.data_preparation_funcs

In [ ]:
import mpunet.preprocessing.data_preparation_funcs
train, val = data_preparation_funcs.get_sequencers(train_queue, val_queue, logger, hparams)

In [ ]:
import importlib
import sys

In [ ]:
importlib.reload(sys.modules['mpunet.preprocessing.data_preparation_funcs'])
from mpunet.preprocessing.data_preparation_funcs import get_sequencers
train, val = get_sequencers(train_queue, val_queue, logger, hparams)

In [ ]:
importlib.reload(sys.modules['mpunet.sequences.isotrophic_live_view_sequence_2d'])

In [ ]:
[i for i in train]

In [ ]:
train.batch_size = 6

train[0]

In [ ]:
train.__getitem__(0)